# 07 Experiment Tracking Class

In [4]:
from helper_functions import plot_predictions, plot_decision_boundary, accuracy_fn
from going_modular import engine, data_loaders
import mlxtend
from mlxtend.plotting import plot_confusion_matrix
import numpy as np
import os
import pandas as pd
from pathlib import Path
import random
import requests
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from torchinfo import summary
import torch
from torch import nn
from torchmetrics import Accuracy, ConfusionMatrix
import torchvision
from torchvision import datasets

from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import Tuple, Dict, List
import zipfile



In [5]:
# see torch and torch vision plus
print (torch.__version__)
print (torchvision.__version__)

2.3.1+cu121
0.18.1+cu121


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def set_seeds (seed: int=42):
    """_summary_

    Args:
        seed (int, optional): _description_. Defaults to 42.
    """ 
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)   

In [7]:
# do helder function
def download_data (source: str,
                   destination: str,
                   data_dir: str = '../data',
                   remove_source: bool = True) -> Path:
    """_summary_

    Args:
        soure (str): _description_
        destination (str): _description_
        remove_source (bool, optional): _description_. Defaults to True.

    Returns:
        Path: _description_
    """    

    data_path = Path (data_dir)
    image_path = data_path / destination
    if image_path.is_dir():
        print (f' {image_path} directory exists skipping download')
    else:
        print (f'[INFO] creating {image_path}')
        image_path.mkdir(parents = True, exist_ok = True)
        target_file = Path(source).name
        with open(data_path/ target_file, 'wb') as f:
            request = requests.get(source)
            print (f'[INFO] downloading {target_file} from {source}')
            f.write (request.content)
        with zipfile.ZipFile(data_path/target_file, 'r') as zip_ref:
            print (f'[INFO] Unsipping {target_file}')
            zip_ref.extractall(image_path)

        if remove_source:
            os.remove (data_path/target_file)
   
    return image_path


In [ ]:
# data_path = Path('../data')
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi_1")

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [13]:
# manual data loader
image_path = Path('../data/pizza_steak_sushi_1')
train_dir = image_path /'train'
test_dir = image_path/'test'

normalize = transforms.Normalize(mean =[0.485, 0.456, 0.406],
                                 std = [0.229, 0.224, 0.225])
manual_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize
])

train_dataloader, test_dataloader, class_names = data_loaders.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = manual_transforms,
    batch_size = 32
)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x21282498b80>,
 ['pizza', 'steak', 'sushi'])

In [14]:
# auto data laoder
image_path = Path('../data/pizza_steak_sushi_1')
train_dir = image_path /'train'
test_dir = image_path/'test'

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
automatic_transforms = weights.transforms()
print (f'Auto transforms: {automatic_transforms}')

train_dataloader, test_dataloader, class_names= data_loaders.create_dataloaders(
    train_dir=train_dir,
    test_dir = test_dir,
    transform=automatic_transforms,
    batch_size = 32
)

train_dataloader, test_dataloader, class_names

Auto transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x212836f3fd0>,
 ['pizza', 'steak', 'sushi'])

## get pretrained model.



In [15]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model_B0 = torchvision.models.efficientnet_b0(weights = weights).to(device)
model_B0

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Hassan/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:01<00:00, 12.9MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [18]:
for param in model_B0.features.parameters():
    param.requires_grad = False
    
set_seeds()

model_B0.classifier= torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features = len(class_names),
              bias = True).to(device)
)

In [19]:
from torchinfo import summary
summary(model_B0,
        input_size = (32, 3, 224, 224),
        verbose = 0,
        col_names = ['input_size', 'output_size', 'num_params','trainable' ],
        col_width = 20,
        row_settings = ['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

## Train model